In [50]:
!pip install datasets
!pip install openai
!pip install llmlingua

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [52]:
import numpy
from datasets import load_dataset

gsm8k = load_dataset('gsm8k', 'main')
validation_index = numpy.load('./day_1/prompts/validation_index.npy')
validation_data = gsm8k['train'].select(validation_index)
gsm8k_test = gsm8k['test']

In [53]:
prompt_hardest = open('./day_1/prompts/prompt_hardest.txt').read()
prompt_with_q = prompt_hardest + '\nQuestion: ' + gsm8k_test[1]['question'] + '\n'

In [54]:
print(prompt_with_q)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [55]:
from openai import OpenAI

client = OpenAI()
gpt_model = "gpt-4o-mini"

In [56]:
completion_without_compression = client.chat.completions.create(
    model=gpt_model,
    messages=[
        {"role": "system", "content": "Follow the given examples and answer the question."},
        {"role": "user", "content": prompt_with_q},
    ]
)


In [57]:
from llmlingua import PromptCompressor

llm_lingua = PromptCompressor(
    model_name="microsoft/llmlingua-2-bert-base-multilingual-cased-meetingbank",
    use_llmlingua2=True,
    device_map="cpu",
)
compressed_prompt = llm_lingua.compress_prompt(prompt_hardest, target_token=300)
compressed_prompt_with_q = compressed_prompt['compressed_prompt'] + '\nQuestion: ' + gsm8k_test[1]['question'] + '\n'

Token indices sequence length is longer than the specified maximum sequence length for this model (2209 > 512). Running this sequence through the model will result in indexing errors


In [58]:
print(compressed_prompt_with_q)

Angelo Melanie hours 2 chapters 4 worksheets 3 hours 1. 5 hours 4 hours break 30 minutes lunch 3 hours 1. 5 hours 12 hours breaks 30 minutes 12 hours 4 days Mark ' s team scores 25 2 pointers 8 10 free throws total points scores 25 2 pointersteam scores 6 3 pointers 8 24 10 10 scored 50 84 points opponents double scored half scored 100 117 points 84 + 117 = 201 Bella two times marbles 20 frisbees 2 / 5 * 60 = 24 30 42 20 frisbeesdeck cards 10 + 4 14 14 + 42 84 140 9 apples 15 oranges 14 bananas 2 9 + 15 24 + 3 114 9 - 2 7 15 13 13 + 7 20 14 - 12 bananas 20 32 4 apples 36 $ 0 50price watermelon W 4 apples 1W = 4A 12 watermelons $ 60 $ 60 = 12W convert $ 60 = one apple $ 60 = $ 1 Susy 800 Sarah 300 100 40 third 90 100 140 40 20 20 10 Sarah 50 30 10 180 180 boxes $ 3 $ 2profit bought 12 120 360 5 sold $ 15 360 330 110 $ 2 220 earned 220 235 115 2 / 3 1 5 3 : 2 3 2 5 females 1000 200 3 = 600 males 2 400 females 480 520 520
Question: A robe takes 2 bolts of blue fiber and half that much whi

In [59]:
completion_with_compression = client.chat.completions.create(
    model=gpt_model,
    messages=[
        {"role": "system", "content": "Follow the given examples and answer the question."},
        {"role": "user", "content": compressed_prompt_with_q},
    ]
)


In [60]:
print(completion_without_compression.choices[0].message.content)

Let's think step by step.

A robe requires 2 bolts of blue fiber. 

For the white fiber, it takes half the amount of the blue fiber, which is 2 bolts / 2 = 1 bolt of white fiber.

To find the total number of bolts required for the robe, we add the bolts of blue and white fiber together:

Total bolts = 2 bolts of blue + 1 bolt of white = 3 bolts in total.

The answer is 3.


In [61]:
print(completion_with_compression.choices[0].message.content)

To make one robe, it takes 2 bolts of blue fiber and half that much white fiber. 

Half of 2 bolts is 1 bolt of white fiber. 

So, the total number of bolts needed is:
2 bolts (blue) + 1 bolt (white) = 3 bolts in total. 

Therefore, it takes 3 bolts in total to make one robe.
